# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [1]:
import pandas as pd
# data = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
data = pd.read_csv('./raw_data.csv')
data.head(5)

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106


In [2]:
data.Data_Value /= 10.0

In [3]:
firstday_2015 = pd.to_datetime('2015-01-01')

In [4]:
data.Date = pd.to_datetime(data.Date)

data_tenyears = data[(data.Date < firstday_2015) & (data.Date != '2008-02-29') & 
                    (data.Date != '2012-02-29')]
data_2015 = data[data.Date >= firstday_2015]

In [5]:
# data_tenyears['MM-DD'] = data_tenyears.Date.dt.month.astype(str) \
#                        + '-' + data_tenyears.Date.dt.day.astype(str)

data_tenyears['MM-DD'] = data_tenyears.Date.dt.strftime('%m-%d')

df1 = data_tenyears.groupby('MM-DD')['Data_Value'].agg(['min', 'max']).reset_index()   

df1.head(1)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,MM-DD,min,max
0,01-01,-16.0,15.6


In [6]:
data_tenyears.dtypes

ID                    object
Date          datetime64[ns]
Element               object
Data_Value           float64
MM-DD                 object
dtype: object

In [7]:
df1['MM-DD'] = '2015-' + df1['MM-DD']  
df1['MM-DD'] = pd.to_datetime(df1['MM-DD'])
df1 = df1.rename(columns={'MM-DD': 'Date'})

In [8]:
df2 = data_2015.groupby('Date').Data_Value.agg(['min', 'max']).rename(columns={'min':'min_2015', 'max':'max_2015'}).reset_index()
df2.head()
# print(df1.shape, df2.shape)

,Date,min_2015,max_2015
0,2015-01-01,-13.3,1.1
1,2015-01-02,-12.2,3.9
2,2015-01-03,-6.7,3.9
3,2015-01-04,-8.8,4.4
4,2015-01-05,-15.5,2.8


In [9]:
import numpy as np

df1 = pd.merge(df1, df2, how='inner', on = 'Date')


df1['min_2015'] = df1.apply(lambda row: np.nan if (row['min_2015'] >= row['min'])
                            and (row['min_2015'] <= row['max']) else row['min_2015'], axis = 1)
df1['max_2015'] = df1.apply(lambda row: np.nan if (row['max_2015'] >= row['min'])
                            and (row['max_2015'] <= row['max']) else row['max_2015'], axis = 1)

df1 = df1.sort_values('Date')

In [10]:
df1.head()

,Date,min,max,min_2015,max_2015
0,2015-01-01,-16.0,15.6,NaN,NaN
1,2015-01-02,-26.7,13.9,NaN,NaN
2,2015-01-03,-26.7,13.3,NaN,NaN
3,2015-01-04,-26.1,10.6,NaN,NaN
4,2015-01-05,-15.0,12.8,-15.5,NaN


In [1]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker


# plot 2005 - 2014 data by lines
plt.figure(figsize=(15,10))

plt.plot(df1['Date'], df1['max'], label = 'Daily record of highest temperatures \nacross years 2005-2014 ')
plt.plot(df1['Date'], df1['min'], label = 'Daily record of lowest temperatures \nacross years 2005-2014 ')

ax = plt.gca()

ax.fill_between(list(df1['Date']), df1['min'], df1['max'], facecolor='y', alpha=0.2)

# plot 2015 data by scattor plot 
plt.scatter(list(df1['Date']), df1['max_2015'], marker='x', s=25, c='k', label = 'Record-breaking daily high temperature \nin year 2015')
plt.scatter(list(df1['Date']), df1['min_2015'], marker='o', s=25, c='m', label = 'Record-breaking daily low temperature \nin year 2015')

ax.xaxis.set_major_formatter(DateFormatter("%m-%d"))
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
for item in ax.xaxis.get_ticklabels():
    item.set_rotation(45)

ax.tick_params(which='minor', length=4)
ax.tick_params(which='major', length=7)
ax.tick_params(labelsize = 15)

plt.xlabel('Date (MM-DD)', fontsize=18)
plt.ylabel('Temperature (Celsius)', fontsize=18)
plt.legend(fontsize=15)
plt.title('Daily high and low temperatures on a year-on-year basis from 2005 through 2014 \n vs. record-breaking temperatures in 2015',
          fontsize=18)

plt.ylim([-40, 50])
plt.grid()
plt.show()

NameError: name 'df1' is not defined